# Лабораторная работа №2: Распознавание именованых сущностей (NER)

**Выполнил: Артамонов Д.С, 20 МАГ ИАД**

In [3]:
import time
import json
import csv

import torch
from torch import nn
from torchtext.data import Field, BucketIterator, TabularDataset
from torchtext.datasets import SequenceTaggingDataset

Дополнительные пакеты, которые не входят в `requirements.txt`

In [ ]:
# !pip install torchtext==0.6.0 nerus

В качестве датасета будем использовать [Nerus](https://github.com/natasha/nerus) - почти 700к статей из Ленты.ру, собранных для проекта [Natasha](https://github.com/natasha)

## Датасет Nerus и дары torch.text

Датасет загружается отедльно архивчиком: [link](https://github.com/natasha/nerus#:~:text=Download-,nerus_lenta.conllu.gz,-~2GB%2C%20~700K%20texts)

Внтури что-то такое:

```bash
friday@fridaydevice:~/HSE/HSE-NLP$ gunzip -c nerus_lenta.conllu.gz | head
# newdoc id = 0
# sent_id = 0_0
# text = Вице-премьер по социальным вопросам Татьяна Голикова рассказала, в каких регионах России зафиксирована наиболее высокая смертность от рака, сообщает РИА Новости.
1	Вице-премьер	_	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Sing	7	nsubj	_	Tag=O
2	по	_	ADP	_	_	4	case	_	Tag=O
3	социальным	_	ADJ	_	Case=Dat|Degree=Pos|Number=Plur4amod	_	Tag=O
4	вопросам	_	NOUN	_	Animacy=Inan|Case=Dat|Gender=Masc|Number=Plur	1	nmod	_	Tag=O
5	Татьяна	_	PROPN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	1	appos	_	Tag=B-PER
6	Голикова	_	PROPN	_	Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing	5	flat:name	_	Tag=I-PER
7	рассказала	_	VERB	_	Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	Tag=O
```

In [4]:
NERUS = './nerus_lenta.conllu.gz'

Авторы датасета сделали удобное API для работы с этим файликом, импортируем его

In [5]:
from nerus import load_nerus

А ещё импортируем магию RusVectores, которую мы будем использовать для стэмминга

In [6]:
from ufal.udpipe import Model, Pipeline
import webvectors.preprocessing.rus_preprocessing_udpipe as udpipe_preproc # cloned from https://github.com/akutuzov/webvectors/blob/master/preprocessing/rus_preprocessing_udpipe.py

UDPIPE_MODEL = 'udpipe_syntagrus.model'


Loading the model...
Processing input...


In [7]:
class Stemmer:
    def __init__(self,modelfile):
        self.stemming_model = Model.load(modelfile)
        self.process_pipeline = Pipeline(self.stemming_model, 'tokenize',Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')
        
    def stem_word(self, word):
        return udpipe_preproc.process(
            self.process_pipeline, text=udpipe_preproc.unify_sym(word), keep_punct=True, keep_pos=False
        )

> Пример стеминга

С помощью API можно загрузить датасет в одно строчку и получить итератор, который будет отдавать статьи и их разметку

In [ ]:
docs = load_nerus(NERUS)
doc = next(docs)

In [8]:
docs = load_nerus(NERUS)
doc = next(docs)
stemmer = Stemmer(UDPIPE_MODEL)
for i in range(10):
    text = doc.sents[0].tokens[i].text
    res = udpipe_preproc.process(stemmer.process_pipeline, text=udpipe_preproc.unify_sym(text), keep_punct=True, keep_pos=False) 
    print(f"{text} -> {res}")

Вице-премьер -> ['вице-премьер']
по -> ['по']
социальным -> ['социальный']
вопросам -> ['вопрос']
Татьяна -> ['татьяна']
Голикова -> ['голикова']
рассказала -> ['рассказывать']
, -> [',']
в -> ['в']
каких -> ['какой']


Возьмём кусочек датасета и распилим его на train/test/validation и запишем их в `tsv` файлики, а ещё применим стэмминг

In [10]:
def dump_dataset(path, stemmer, train_num = 5_000, val_num = 500, test_num = 1000):
    docs = load_nerus(path)
    for num, file in [(train_num, f"ner_train_{train_num}.tsv"), (val_num,  f"ner_val_{val_num}.tsv"), (test_num,  f"ner_test_{test_num}.tsv")]:
        with open (file, 'w') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            while (num):
                doc = next(docs)
                for sent in doc.sents:
                    for token in sent.tokens:
                        stem_word = stemmer.stem_word(token.text)
                        if stem_word:
                            # ignore symbols, emojies and foereign languages 
                            text = stem_word[0]
                            tsv_writer.writerow(
                                [text,  token.tag]
                            )
                        
                tsv_writer.writerow([]) # empty line between documents
                num -= 1


Запускаем...

( или не запускаем, потому что долго)

In [11]:
# stemmer = Stemmer(UDPIPE_MODEL)
# dump_dataset(path=NERUS, stemmer=stemmer)

А теперь применим магию TorchText, чтобы сделать удобный DataLoader. В старой версии TorchText есть удобный `SequenceTaggingDataset`, который сделает нам из файликов DataLoader

In [12]:
word_field = Field(lower=True)
tag_field = Field(unk_token=None)
train_data, val_data, test_data = SequenceTaggingDataset.splits(
    path='.',
    train='ner_train_5000.tsv',
    validation='ner_val_500.tsv',
    test='ner_test_1000.tsv',
    fields=(('word', word_field),  ('tag', tag_field))
)

Каждый элемент датасетов представляет собой 2 списка: 
* список с предобработанными словами статьи (и знакам пунктуации)
* список тегов, соответствующих им

При этом каждый элемент - отдельная новостная статья из датасета: `SequenceTaggingDataset` умеет разбивать по пустой строке, которую мы оставили между статьями

In [11]:
train_data[333].__dict__.values()

dict_values([['жительница', 'американский', 'город', 'оуингс', 'милс', ',', 'штат', 'мэриленд', ',', 'выигрывать', 'джекпот', 'в', 'несколько', 'тысяча', 'доллар', '.', 'как', 'сообщать', 'информационный', 'портал', 'upi', ',', 'она', 'благодарить', 'за', 'это', 'бессонница', '.', 'работать', 'медсестра', '72-летняе', 'женщина', 'решать', 'ложиться', 'рано', 'перед', 'долгий', 'смена', 'в', 'больница', ',', 'но', 'в', 'она', 'не', 'выходить', '.', '""""', 'я', 'постоянно', 'смотреть', 'на', 'час', 'и', 'беситься', ',', 'что', 'никак', 'не', 'мочь', 'засыпать', '.', 'последний', 'время', ',', 'который', 'я', 'запомнить', ',', 'быть', 'xx', ':', 'xx', 'вечер', '""""', ',', '-', 'рассказывать', 'американка', '.', 'по', 'она', 'слово', ',', 'этот', 'число', 'запасть', 'она', 'в', 'голова', ',', 'поэтому', 'на', 'следующий', 'после', 'смена', 'день', 'она', 'отправляться', 'в', 'магазин', 'за', 'лотерейный', 'билет', '.', 'женщина', 'решать', 'поиграть', 'в', 'виртуальный', 'скачка', 'и', '

Теперь составим словари слов и токен с помощью `Field` класса, а ещё сохраним индексы "пустого" слова и тэга. Этими "пустыми" словами и тэгами будут "дозабиваться" матрички документа в батче, чтобы каждый документ из батча был одного размера

In [18]:
# convert fields to vocabulary list
min_word_freq = 3
batch_size = 10
word_field.build_vocab(train_data.word, min_freq=min_word_freq)
tag_field.build_vocab(train_data.tag)

# prepare padding index to be ignored during model training/evaluation
word_pad_idx = word_field.vocab.stoi[word_field.pad_token]
tag_pad_idx = tag_field.vocab.stoi[tag_field.pad_token]

# # create iterator for batch input
train_iter, val_iter, test_iter = BucketIterator.splits(
    datasets=(train_data, val_data, test_data),
    batch_size=batch_size
)


In [19]:
print(f"Empty word index: {word_pad_idx}")
print(f"Empty tag index: {tag_pad_idx}")

Empty word index: 1
Empty tag index: 0


In [20]:
print(f"Train set: {len(train_data)} sentences")
print(f"Val set: {len(val_data)} sentences")
print(f"Test set: {len(test_data)} sentences")

Train set: 5000 sentences
Val set: 500 sentences
Test set: 1000 sentences


Чтобы посмотреть, что в батче, можно раскомментровать и запустить код ниже:

In [38]:
# for batch in train_iter:
#     print(batch)
#     print(batch.word)
#     print(batch.tag)
#     break

Обернём всё в один класс, чтобы было проще его перекладывать туда-сюда во время тренировки

In [21]:
from dataclasses import dataclass
@dataclass
class DataIterator:
    train_iter: BucketIterator
    val_iter: BucketIterator
    test_iter: BucketIterator
    tag_pad_idx: int
    word_pad_idx: int

## Моделька

В этот раз  натренируем свои эмбэдинги и засунем их в bidirectional LSTM.

Моделька будет состоять из:
1. Embedding слоя ([nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html))
1. BiLSTM слоёв ([nn.LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html))
1. Линейного слоя, чтобы сделать классификацию


In [22]:
class BiLSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, lstm_layers,
               emb_dropout, lstm_dropout, fc_dropout, word_pad_idx):
        super().__init__()
        self.embedding_dim = embedding_dim

        # LAYER 1: Embedding layer
        self.embedding = nn.Embedding(
            num_embeddings=input_dim, 
            embedding_dim=embedding_dim, 
            padding_idx=word_pad_idx
        )
        self.emb_dropout = nn.Dropout(emb_dropout) # TODO: we can add dropout in v2
        # LAYER 2: BiLSTM layer
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=lstm_layers,
            bidirectional=True,
            dropout=lstm_dropout if lstm_layers > 1 else 0
        )
        # LAYER 3: Fully-connected layer
        self.fc_dropout = nn.Dropout(fc_dropout)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # times 2 for bidirectional

    def forward(self, sentence):
        # sentence = [sentence length, batch size]
        # embedding_out = [sentence length, batch size, embedding dim]
        embedding_out = self.emb_dropout(self.embedding(sentence))
        # lstm_out = [sentence length, batch size, hidden dim * 2]
        lstm_out, _ = self.lstm(embedding_out)
        # ner_out = [sentence length, batch size, output dim]
        ner_out = self.fc(self.fc_dropout(lstm_out))
        return ner_out

    def init_weights(self):
        # to initialize all parameters from normal distribution
        # helps with converging during training
        for name, param in self.named_parameters():
            nn.init.normal_(param.data, mean=0, std=0.1)

    def init_embeddings(self, word_pad_idx):
        # initialize embedding for padding as zero
        self.embedding.weight.data[word_pad_idx] = torch.zeros(self.embedding_dim)

    def count_parameters(self):
        return sum(p.numel() for p in self.parameters() if p.requires_grad)

In [23]:
bilstm = BiLSTM(
    input_dim=len(word_field.vocab),
    embedding_dim=120, # 300
    hidden_dim=64,
    output_dim=len(tag_field.vocab),
    lstm_layers=2,
    emb_dropout=0.5,
    lstm_dropout=0.1,
    fc_dropout=0.25,
    word_pad_idx=word_pad_idx
)

In [24]:
bilstm.init_weights()
bilstm.init_embeddings(word_pad_idx=word_pad_idx)
print(f"The model has {bilstm.count_parameters():,} trainable parameters.")
print(bilstm)

The model has 2,323,192 trainable parameters.
BiLSTM(
  (embedding): Embedding(17730, 120, padding_idx=1)
  (emb_dropout): Dropout(p=0.5, inplace=False)
  (lstm): LSTM(120, 64, num_layers=2, dropout=0.1, bidirectional=True)
  (fc_dropout): Dropout(p=0.25, inplace=False)
  (fc): Linear(in_features=128, out_features=8, bias=True)
)


## Тренировка

Ниже будет много функций для тренировки и валидации модельки

In [26]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


(Хорошо, что cuda отвалилась после тренировки, а не до 🙃) 

Простая функция, что считать время одной эпохи

In [20]:
  def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Метрика качества классификации. Будет использовать обычную accuracy, но с дополнительными фишками для учёта паддингов

In [21]:
def accuracy(preds, y, device):
    max_preds = preds.argmax(dim=1, keepdim=True).to(device)  # get the index of the max probability
    non_pad_elements = (y != data.tag_pad_idx).nonzero().to(device)  # prepare masking for paddings
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements]).to(device)
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]]).to(device)

Функция для запуска одной эпохи, всё довольно стандартно

In [22]:
def run_epoch(model, data, optimizer, loss_fn, device):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    for batch in data.train_iter:
        # text = [sent len, batch size]
        text = batch.word.to(device)
        # tags = [sent len, batch size]
        true_tags = batch.tag.to(device)
        optimizer.zero_grad()
        pred_tags = model(text)
        # to calculate the loss and accuracy, we flatten both prediction and true tags
        # flatten pred_tags to [sent len, batch size, output dim]
        pred_tags = pred_tags.view(-1, pred_tags.shape[-1])
        # flatten true_tags to [sent len * batch size]
        true_tags = true_tags.view(-1)
        batch_loss = loss_fn(pred_tags, true_tags)
        batch_acc = accuracy(pred_tags, true_tags, device)
        batch_loss.backward()
        optimizer.step()
        epoch_loss += batch_loss.item()
        epoch_acc += batch_acc.item()
    return epoch_loss / len(data.train_iter), epoch_acc / len(data.train_iter)

Функция для валидации

In [23]:
def evaluate(model, iterator, loss_fn, device):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    with torch.no_grad():
      # similar to epoch() but model is in evaluation mode and no backprop
        for batch in iterator:
            text = batch.word.to(device)
            true_tags = batch.tag.to(device)
            pred_tags = model(text)
            pred_tags = pred_tags.view(-1, pred_tags.shape[-1])
            true_tags = true_tags.view(-1)
            batch_loss = loss_fn(pred_tags, true_tags)
            batch_acc = accuracy(pred_tags, true_tags, device)
            epoch_loss += batch_loss.item()
            epoch_acc += batch_acc.item()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

И, наконец, тренировка 

In [24]:
def train(model, data, optimizer, loss_fn, device, n_epochs, n_val=5):
    for epoch in range(n_epochs):
        start_time = time.time()
        train_loss, train_acc = run_epoch(model, data, optimizer, loss_fn,device)
        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        print(f"Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s")
        print(f"\tTrn Loss: {train_loss:.3f} | Trn Acc: {train_acc * 100:.2f}%")
        if epoch % n_val == 0:
            val_loss, val_acc = evaluate(model, data.val_iter, loss_fn, device)
            print(f"\tVal Loss: {val_loss:.3f} | Val Acc: {val_acc * 100:.2f}%")
    test_loss, test_acc = evaluate(model, data.test_iter, loss_fn, device)
    print(f"Test Loss: {test_loss:.3f} |  Test Acc: {test_acc * 100:.2f}%")

Инициализируем и запускаем

In [27]:
model=bilstm.to(device)
data= DataIterator(train_iter=train_iter, val_iter=val_iter, test_iter=test_iter,tag_pad_idx=tag_pad_idx,word_pad_idx=word_pad_idx)
optimizer=torch.optim.Adam(model.parameters())
loss_fn=nn.CrossEntropyLoss(ignore_index=data.tag_pad_idx)

In [29]:
train(model, data, optimizer, loss_fn, device, n_epochs=10)

Epoch: 01 | Epoch Time: 1m 16s
	Trn Loss: 0.313 | Trn Acc: 91.71%
	Val Loss: 0.118 | Val Acc: 96.33%
Epoch: 02 | Epoch Time: 1m 14s
	Trn Loss: 0.100 | Trn Acc: 96.88%
Epoch: 03 | Epoch Time: 1m 14s
	Trn Loss: 0.068 | Trn Acc: 97.86%
Epoch: 04 | Epoch Time: 1m 15s
	Trn Loss: 0.055 | Trn Acc: 98.26%
Epoch: 05 | Epoch Time: 1m 16s
	Trn Loss: 0.046 | Trn Acc: 98.53%
Epoch: 06 | Epoch Time: 1m 17s
	Trn Loss: 0.041 | Trn Acc: 98.70%
	Val Loss: 0.060 | Val Acc: 98.14%
Epoch: 07 | Epoch Time: 1m 18s
	Trn Loss: 0.035 | Trn Acc: 98.87%
Epoch: 08 | Epoch Time: 1m 15s
	Trn Loss: 0.032 | Trn Acc: 98.95%
Epoch: 09 | Epoch Time: 1m 15s
	Trn Loss: 0.029 | Trn Acc: 99.06%
Epoch: 10 | Epoch Time: 1m 16s
	Trn Loss: 0.027 | Trn Acc: 99.13%
Test Loss: 0.075 |  Test Acc: 97.93%


In [30]:
torch.save(model.state_dict(), 'ner-bilstm.pt')

Кажется, что-то даже получилось 🤓 Тестовая accuracy 97.93% 